In [61]:
import pandas as pd
from typing import Union


In [93]:
pd.set_option('display.max_columns', None)        # don’t omit any columns
pd.set_option('display.width', None)              # no wrapping at a fixed terminal width
pd.set_option('display.max_colwidth', None)       # show full contents of each cell

votes_df   = pd.read_csv(r'const\votes.csv')
members_df = pd.read_csv(r'const\mk_individual.csv')
members_df[members_df['mk_individual_name'] == 'נתניהו']


,mk_individual_id,mk_status_id,mk_individual_name,mk_individual_name_eng,mk_individual_first_name,mk_individual_first_name_eng,mk_individual_email,mk_individual_photo,PersonID,LastName,FirstName,GenderID,GenderDesc,Email,IsCurrent,LastUpdatedDate,altnames
145,90,0,נתניהו,NaN,בנימין,NaN,bnetanyahu@knesset.gov.il,https://oknesset.org/static/img/Male_portrait_placeholder_cropped.jpg,965,נתניהו,בנימין,251,זכר,bnetanyahu@knesset.gov.il,True,2022-11-10 11:19:09,"[""בנימין נתניהו"", ""בינימין נתניהו""]"


In [96]:
def get_person_id(df: pd.DataFrame, first_name: str, last_name: str):
    """
    Return the PersonID for a given first and last name.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing at least the columns 'FirstName', 'LastName', and 'PersonID'.
    first_name : str
        The first name to look up.
    last_name : str
        The last name to look up.

    Returns:
    --------
    The matching PersonID (as stored in df['PersonID']), or:
      - None if no match is found,
      - A list of PersonIDs if multiple matches are found.
    """
    # Filter rows matching both first and last name (case-sensitive)
    matches = df[
        (df['FirstName'] == first_name) &
        (df['LastName']  == last_name)
    ]

    if matches.empty:
        return None
    elif len(matches) == 1:
        return matches.iloc[0]['mk_individual_id']
    else:
        # Multiple matches: return all matching IDs
        return matches['mk_individual_id'].tolist()
# example run for Yair Lapid
person_id = get_person_id(members_df, 'בנימין', 'נתניהו')
print(f"Person ID for בנימין נתניהו {person_id}")

Person ID for בנימין נתניהו 90


In [101]:
def filter_votes_by_mk(df: pd.DataFrame, mk_id: Union[int, str]) -> pd.DataFrame:
    """
    Return only the rows where mk_id is in mk_ids_pro or mk_ids_against,
    keep only the columns: sess_item_dscr, vote_item_id, vote_item_dscr, vote_date, vote_time,
    and add two flags: voted_for and voted_against.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing at least 'mk_ids_pro', 'mk_ids_against',
        and the columns to select after filtering.
    mk_id : int or str
        The member ID to filter by.

    Returns
    -------
    pd.DataFrame
        Subset of df where mk_id appears in mk_ids_pro or mk_ids_against,
        with the columns:
          ['sess_item_dscr', 'vote_item_id', 'vote_item_dscr',
           'vote_date', 'vote_time', 'voted_for', 'voted_against']
    """
    def contains_id(cell):
        if isinstance(cell, (list, set)):
            return mk_id in cell
        # split comma‑separated strings (handles None/NaN via str())
        return str(mk_id) in [item.strip() for item in str(cell).split(',') if item.strip()]

    mask_pro     = df['mk_ids_pro'].apply(contains_id)
    mask_against = df['mk_ids_against'].apply(contains_id)

    # filter and copy to avoid SettingWithCopyWarning
    filtered = df.loc[mask_pro | mask_against].copy()

    # add the vote flags
    filtered['voted_for']    = mask_pro.loc[filtered.index].astype(int)
    filtered['voted_against'] = mask_against.loc[filtered.index].astype(int)
    # convert sess_item_dscr to string
    filtered['sess_item_dscr']  = filtered['sess_item_dscr'].astype('string')
    filtered['vote_item_dscr']  = filtered['vote_item_dscr'].astype('string')
    return filtered[
        ['sess_item_dscr',
         'vote_item_id',
         'vote_item_dscr',
         'vote_date',
         'vote_time',
         'voted_for',
         'voted_against']
    ]
    
# example run for Yair Lapid
filtered_votes = filter_votes_by_mk(votes_df, 90)
# Filter rows where sess_item_dscr is <NA> or null
filtered_votes = filtered_votes[filtered_votes['sess_item_dscr'].notna()]
filtered_votes.to_csv(r'const\filtered_votes.csv', index=False, encoding='utf-8-sig')




In [102]:
alist = [
    'הודעת ראש הממשלה על פעילות הממשלה בתקופה שחלפה ועל תוכניותיה למושב הקרוב',
    'הודעת יושב-ראש ועדת הכנסת',
    'הצעת חוק גיל פרישה, התשס"ד-2003',
    'הצעת חוק המדיניות הכלכלית לשנת הכספים 2004 (תיקוני חקיקה), התשס"ד-2003 (קריאה שניה וקריאה שלישית) - גמר הדיון והצבעות',
    'הצעת חוק הדיינים (תיקון מס` 19), התשס"ד-2004',
    'הצ"ח הכנסת (תיקון מס` 18), התשס"ד-2004',
    'הצעת חוק הבחירות לכנסת (תיקון מס` 51), התשס"ד-2004',
    'התבטאויותיו של הרב אביגדור נבנצאל והתבטאות ראש השב"כ בהקשר להתנגדות לתכנית ההתנתקות',
    'השיטה הנהוגה בבתי-הספר להפצת "תרופות התנהגותיות"',
    'שחרור אסירים פלשתינים',
    'הבנות ג`נבה',
    'הצעת חוק יישום תכנית ההתנתקות, התשס"ה-2005',
    'הצעת חוק הרשויות המקומיות (מימון בחירות) (תיקון מס` 6)(הארכת מועדים בבחירות מיוחדות - הוראת שעה), התשס"ה-2005',
    'הצעת חוק האזרחות והכניסה לישראל (הוראת שעה)(תיקון), התשס"ה-2005',
    'הצעת חוק להגברת התחרות ולצמצום הריכוזיות וניגודי הענינים בשוק ההון בישראל (תיקוני חקיקה), התשס"ה-2005',
    'הצעת חוק לתיקון פקודת מס הכנסה (מס` 147), התשס"ה-2005',
    'הצעת חוק התפזרות הכנסת השש-עשרה, התשס"ג-2003',
    'הצעת חוק המפלגות (מימון בחירות מקדימות לכנסת השבע עשרה) (הוראות שעה), התשס"ו - 2005',
    'דו"ח ועדת החוקה, חוק ומשפט בעניין הצעת חוקה למדינת ישראל - דיון',
    'הצעת ועדת הכנסת להקים ועדת חקירה פרלמנטרית לאירועים שהתרחשו בעמונה',
    'הצעת חוק הסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב והמדיניות הכלכלית לשנת הכספים 2006), התשס"ו-2006',
    'הצעת חוק התקציב לשנת הכספים 2006, התשס"ו-2006',
    'הצעת חוק הרשות הלאומית לבטיחות בדרכים (הוראת שעה), התשס"ו-2006',
    'חוק ההסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב והמדיניות הכלכלית לשנת הכספים 2007), התשס"ז-2007',
    'חוק משק החשמל (תיקון מס` 5), התשס"ז - 2007',
    'הצעת חוק הגבלת העישון במקומות ציבוריים (תיקון מס` 2), התשס"ז-2007',
    'הצעת חוק ההסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב והמדיניות הכלכלית לשנת הכספים 2008), התשס"ח-2007',
    'הצעת חוק שכר מינימום (העלאת סכומי שכר מינימום - הוראת שעה) (תיקון מס` 2), התשס"ח-2007',
    'הצעת חוק הביטוח הלאומי (תיקון מס` 103), התשס"ח-2007',
    'הצעת חוק העמותות (תיקון מס` 11) (דיווח על תרומות)',
    'הצעת חוק תכנון ובניה (תיקון מס` 120 והוראת שעה), התשע"ו-2015',
    'הצעת חוק התקציב לשנות התקציב 2015 ו-2016, התשע"ו-2015',
    'חוק שירות ביטחון (תיקון מס` 21), התשע"ו-2015',
    'חוק התכנית הכלכלית (תיקוני חקיקה ליישום המדיניות הכלכלית לשנת התקציב 2015 ו-2016), התשע"ו-2015',
    'חוק ההתייעלות הכלכלית (תיקוני חקיקה להשגת יעדי התקציב לשנת התקציב 2015 ו-2016), התשע"ו-2015',
    'חוק מעמדן של ההסתדרות הציונית העולמית ושל הסוכנות היהודית לארץ-ישראל (תיקון מס` 2), התשע"ו–2015',
    'חוק סמכויות לשם שמירה על ביטחון הציבור (תיקון מס` 5 והוראת שעה), התשע"ו-2016',
    'חוק למניעת הסתננות (עבירות ושיפוט) (הוראת שעה), התשע"ו-2016',
    'חוק חובת גילוי לגבי מי שנתמך על ידי ישות מדינית זרה (תיקון), התשע"ו-2016',
    'חוק התכנון והבנייה (תיקון מס` 107 והוראת שעה), התשע"ו-2016',
    'הצעת חוק הפיקוח על שירותים פיננסיים (תיקוני חקיקה), התשע"ו-2016',
    'הצעת חוק התקציב לשנות הכספים 2017 ו-2018, התשע"ז-2016',
    'הצעת חוק התכנית הכלכלית (תיקוני חקיקה ליישום המדיניות הכלכלית לשנות התקציב 2017 ו-2018), התשע"ז-2016',
    'הצעת חוק ההתייעלות הכלכלית (תיקוני חקיקה להשגת יעדי התקציב לשנות התקציב 2017 ו-2018), התשע"ז-2016',
    'הצעת חוק התכנון והבנייה (תיקון מס` 116), התשע"ז-2017',
    'חוק גיל פרישה (הורה שילדו נפטר) (הוראת שעה), התשע"ח-2017',
    'חוק למניעת הסתננות (עבירות ושיפוט) (הוראת שעה), התשע"ח-2017',
    'חוק קרן קיימת לישראל (תחולת דיני המס והוראת שעה), התשע"ח-2018',
    'חוק המפלגות (תיקון מס` 24), התשע"ח-2018',
    'חוק-יסוד: ירושלים בירת ישראל (תיקון מס` 2)',
    'חוק לתיקון דיני הרשויות המקומיות (חוקי עזר בעניין פתיחתם וסגירתם של עסקים בימי מנוחה), התשע"ח-2018',
    'חוק-יסוד: הממשלה (תיקון מס` 5)',
    'חוק הסכמים לנשיאת עוברים (אישור הסכם ומעמד היילוד) (תיקון מס` 2), התשע"ח-2018',
    'חוק-יסוד: ישראל – מדינת הלאום של העם היהודי',
    'חוק גנים לאומיים, שמורות טבע, אתרים לאומיים ואתרי הנצחה (תיקון מס` 17), התשע"ט-2018',
    'חוק הבחירות לכנסת העשרים ושלוש (הוראות מיוחדות), התש"ף?2019',
    'חוק-יסוד: הממשלה (תיקון מס` 8 והוראת שעה)',
    'חוק סמכויות מיוחדות להתמודדות עם נגיף הקורונה החדש (הוראת שעה), התש"ף-2020',
    'הצעת חוק-יסוד: הכנסת (תיקון מס` 50 ? הוראת שעה)',
    'הצעת חוק-יסוד: הכנסת (תיקון מס` 51)'
]




In [103]:
def get_vote_result_by_name(members_df, votes_df, first_name, last_name, vote_titles):
    """
    Get the filtered votes for a given member of parliament and a list of vote titles.

    Parameters:
    -----------
    members_df : pd.DataFrame
        DataFrame containing member details.
    votes_df : pd.DataFrame
        DataFrame containing vote details.
    first_name : str
        The first name of the member.
    last_name : str
        The last name of the member.
    vote_titles : list of str
        List of vote titles to filter.

    Returns:
    --------
    pd.DataFrame
        Filtered DataFrame containing votes matching the given titles.
    """
    id = get_person_id(members_df, first_name, last_name)
    if id is None:
        print(f"Person {first_name} {last_name} not found.")
        return None
    
    # First filter by member ID
    filtered_votes = filter_votes_by_mk(votes_df, id)
    
    # Then return all rows that match any title in vote_titles
    return filtered_votes[filtered_votes['sess_item_dscr'].isin(vote_titles)]

miri_100 = get_vote_result_by_name(members_df,votes_df, 'איתמר', 'בן גביר', alist)
miri_100.to_csv(r'const\bibi_100.csv', index=False, encoding='utf-8-sig')
     

In [ ]:
top100_bils = pd.read_csv(r'const\Top_100_Bills.csv')
